## BigQuery Sample Codes  
AIDP 분석환경에서 제공하는 JupyerHub에서 skt 파이썬 패키지를 사용하면 BigQuery와 연동하여 분석 및 모델링 작업을 수행할 수 있습니다.  
skt 패키지는 분석환경에 기본적으로 설치되어 있으며 pip를 사용하여 버전 업그레이드를 할 수 있습니다.
~~~bash
$ pip install --upgrade skt
~~~

### BigQuery 실행  
다음과 같이 2가지 방법으로 쿼리를 실행하고 결과를 확인할 수 있습니다.  
- IPython Magic을 사용하여 Jupyter Notebook Cell에서 SQL 실행
- BigQuery Client 사용하여 SQL 실행

#### IPython Magic으로 SQL 실행  
Jupyter Notebook Cell에서 SQL을 실행하고 그 결과를 확인할 수 있습니다. 쿼리 결과는 변수에 Pandas Dataframe으로 저장할 수 있습니다. Ditonary로 정의된 쿼리 파라미터를 SQL에 주입할 수 있습니다.

먼저, IPython Magic을 로드합니다.

In [1]:
from skt.gcp import load_bigquery_ipython_magic

load_bigquery_ipython_magic()

다음과 같이 SQL을 실행합니다.

In [2]:
%%bq
    SELECT MAX(dt) as max_dt
    FROM tworld.twd_dst_product_group

BigQuery execution took 2 seconds.


,max_dt
0,2020-07-22


SQL 결과를 Pandas DataFrame으로 변수에 저장할 수 있습니다. 다음은 max_dt라는 변수에 SQL 결과를 저장하는 예제입니다.

In [3]:
from skt.ye import get_spark
from skt.gcp import load_bigquery_ipython_magic, \
                    bq_to_pandas, \
                    get_bigquery_client

load_bigquery_ipython_magic()

In [4]:
%%bq max_dt

SELECT MAX(dt) as value
FROM tworld.twd_dst_product_group

BigQuery execution took 2 seconds.


In [5]:
max_dt

,value
0,2020-07-22


쿼리 파라미터를 SQL에 전달할 수 있습니다. 전달할 값은 Dictionary 타입으로 정의합니다.

In [6]:
query_params = {
    "max_dt": max_dt["value"][0].strftime("%Y-%m-%d")
}

query_params

{'max_dt': '2020-07-22'}

다음과 같이 params 옵션에 전달할 쿼리 파라미터 변수를 정의합니다.

In [7]:
%%bq --params $query_params

SELECT *
FROM tworld.twd_dst_product_group
WHERE dt = '{max_dt}'


BigQuery execution took 2 seconds.


,product_grp_id,product_grp_nm,dt
0,000003412,노키아 바나나폰,2020-07-22
1,000001572,BlackBerry PRIV,2020-07-22
2,000002832,Re 아이폰 8,2020-07-22
3,000002833,Re 아이폰 8 플러스,2020-07-22
4,000002834,Re 아이폰 X,2020-07-22
...,...,...,...
359,000002072,갤럭시 와이드2,2020-07-22
360,000003332,애플워치4 44mm,2020-07-22
361,000004234,갤럭시 S20 플러스 5G,2020-07-22
362,000002412,엘지 스마트 폴더,2020-07-22


#### BigQuery Client로 SQL 실행  
BigQuery Client는 Google에서 제공하는 Client Library에서 제공하는 객체입니다. SQL을 실행할 수 있을 뿐만 아니라 Google Client Library에서 제공하는 다양한 기능들을 활용할 수 있습니다. 더욱 자세한 내용은 <a href="https://googleapis.dev/python/bigquery/latest/generated/google.cloud.bigquery.client.Client.html#google.cloud.bigquery.client.Client" target="_blank">이곳</a>에서 확인하시기 바랍니다.

다음과 같이 BigQuery Client를 생성하여 SQL을 실행합니다. 쿼리 조회 결과가 있는 경우 Iterator 형태로 결과를 리턴해줍니다.

In [8]:
from skt.gcp import get_bigquery_client

bq_client = get_bigquery_client()

sql = """
    SELECT *
    FROM tworld.twd_dst_product_group
    WHERE dt = '{max_dt}'
""".format(**query_params)

bq_client.query(sql).result()

### BigQuery to Pandas
위에서 설명했듯이 BigQuery 쿼리 결과를 Pandas Dataframe으로 리턴받을 수 있습니다.
위의 IPython Magic 뿐만 아니라 skt 패키지에서 제공하는 메서드를 사용할 수도 있습니다.   
   
(참고 : Jupyter Notebook Cell에서 IPyhon Magic을 사용하여 SQL을 실행하는 경우 그 Cell에는 다른 Python 코드를 추가할 수 없습니다. Cell에 다른 Python 코드를 자유롭게 추가하려면 skt 패키지의 기능을 활용하시기 바랍니다.)


다음은 bq_to_pandas 메서드로 결과를 Pandas Dataframe에 저장하는 예입니다.

In [9]:
from skt.gcp import bq_to_pandas

pd_df = bq_to_pandas("""
    SELECT *
    FROM tworld.twd_dst_product_group
    WHERE dt = (SELECT MAX(dt) FROM tworld.twd_dst_product_group)
""")

pd_df

Downloading: 100%|██████████| 364/364 [00:02<00:00, 126.98rows/s]


,product_grp_id,product_grp_nm,dt
0,000003412,노키아 바나나폰,2020-07-22
1,000001572,BlackBerry PRIV,2020-07-22
2,000002832,Re 아이폰 8,2020-07-22
3,000002833,Re 아이폰 8 플러스,2020-07-22
4,000002834,Re 아이폰 X,2020-07-22
...,...,...,...
359,000002072,갤럭시 와이드2,2020-07-22
360,000003332,애플워치4 44mm,2020-07-22
361,000004234,갤럭시 S20 플러스 5G,2020-07-22
362,000002412,엘지 스마트 폴더,2020-07-22


### Pandas to BigQuery

Pandas Dataframe을 특정 BigQuery 테이블에 저장할 수 있습니다.  
다음과 같이 Pandas Dataframe의 to_gbq 메서드를 사용합니다.

In [10]:
import time

dest_table = f"temp_1d.twd_dst_product_group_{str(int(time.time()))}"
print(f"저장할 테이블 : {dest_table}")

pd_df.to_gbq(dest_table)

get_bigquery_client().query(f"""
    SELECT *
    FROM {dest_table}
""").result()

저장할 테이블 : temp_1d.twd_dst_product_group_1595603924


1it [00:04,  4.06s/it]


데이터가 테이블에 저장되었는지 확인합니다.

In [11]:
query_params =  {"dest_table": dest_table}

In [12]:
%%bq --params $query_params

SELECT *
FROM {dest_table}

BigQuery execution took 2 seconds.


,product_grp_id,product_grp_nm,dt
0,000003412,노키아 바나나폰,2020-07-22 00:00:00+00:00
1,000001572,BlackBerry PRIV,2020-07-22 00:00:00+00:00
2,000002832,Re 아이폰 8,2020-07-22 00:00:00+00:00
3,000002833,Re 아이폰 8 플러스,2020-07-22 00:00:00+00:00
4,000002834,Re 아이폰 X,2020-07-22 00:00:00+00:00
...,...,...,...
359,000002072,갤럭시 와이드2,2020-07-22 00:00:00+00:00
360,000003332,애플워치4 44mm,2020-07-22 00:00:00+00:00
361,000004234,갤럭시 S20 플러스 5G,2020-07-22 00:00:00+00:00
362,000002412,엘지 스마트 폴더,2020-07-22 00:00:00+00:00


### BigQuery to Spark  

BigQuery 데이터를 가져와서 Spark로 처리할 수 있습니다. BigQuery SQL 결과를 Spark Dataframe으로 변환한 후 이어서 데이터 처리가 가능합니다.

In [13]:
from skt.gcp import bq_to_df

spark_df = bq_to_df(f"""
    SELECT *
    FROM {dest_table}
""")

spark_df.head(5)

[Row(product_grp_id='000000004', product_grp_nm='GALAXY S4', dt=datetime.datetime(2020, 7, 22, 9, 0)),
 Row(product_grp_id='000004192', product_grp_nm='갤럭시 탭 S6 5G', dt=datetime.datetime(2020, 7, 22, 9, 0)),
 Row(product_grp_id='000004232', product_grp_nm='갤럭시 Z 플립', dt=datetime.datetime(2020, 7, 22, 9, 0)),
 Row(product_grp_id='000003412', product_grp_nm='노키아 바나나폰', dt=datetime.datetime(2020, 7, 22, 9, 0)),
 Row(product_grp_id='000003613', product_grp_nm='iPad mini 5th', dt=datetime.datetime(2020, 7, 22, 9, 0))]

### Spark to BigQuery  

Spark으로 처리한 결과를 다시 BigQuery에 적재가 가능합니다.  
다음은 위에서 spark_df 변수에 저장한 Spark Dataframe을 다시 BigQuery에 적재하는 예제입니다.

1. 먼저 저장할 테이블을 BigQuery에 생성합니다. 여기서는 파티션된 테이블을 생성하고 특정 파티션에 Spark Dataframe을 저장할 것입니다.

In [14]:
from skt.gcp import df_to_bq_table
from pyspark.sql.types import DateType

dest_dataset = "temp_1d"
partitioned_dest_table = f"twd_dst_product_group_{str(int(time.time()))}"

get_bigquery_client().query(f"""
    CREATE OR REPLACE TABLE {dest_dataset}.{partitioned_dest_table}
    (
        product_grp_id STRING,
        product_grp_nm STRING,
        dt DATE
    )
    PARTITION BY dt
""").result()

print(f"생성된 테이블 : {dest_dataset}.{partitioned_dest_table}")

생성된 테이블 : temp_1d.twd_dst_product_group_1595603960


2. Spark Dataframe을 BigQuery 테이블에 저장합니다. 여기서는 파티션 컬럼 타입이 Date이므로 타입 변환 작업을 해주었습니다. 이렇게 경우에 따라 타입 변환 작업이 필요한 경우가 있으므로 타입에 주의해주세요.

In [15]:
changed_df = spark_df.select("product_grp_id", "product_grp_nm", spark_df.dt.cast("date"))
partition_dt = changed_df.head(1)[0].dt.strftime("%Y%m%d")
df_to_bq_table(df=changed_df,
               dataset=dest_dataset,
               table_name=partitioned_dest_table,
               partition=partition_dt,
               mode="overwrite")

3. BigQuery에 저장되었는지 확인합니다.

In [16]:
query_params = {"dataset":dest_dataset, "table_name":partitioned_dest_table}

In [17]:
%%bq --params $query_params
    SELECT *
    FROM {dataset}.{table_name}

BigQuery execution took 3 seconds.


,product_grp_id,product_grp_nm,dt
0,000000004,GALAXY S4,2020-07-22
1,000004192,갤럭시 탭 S6 5G,2020-07-22
2,000004232,갤럭시 Z 플립,2020-07-22
3,000003412,노키아 바나나폰,2020-07-22
4,000003613,iPad mini 5th,2020-07-22
...,...,...,...
359,000002972,갤럭시 와이드3,2020-07-22
360,000004472,서피스 고2 LTE,2020-07-22
361,000003033,엘지 Q7 플러스,2020-07-22
362,000000035,VEGA LTE,2020-07-22
